# Debug Evalscripts locally

This script allows you to more comfortably debug your evalscripts on your local machine using all powerful javascript tools which are conventionally available.

You need to install the following Python packages:

- sentinelhub-py
- esprima

In [1]:
import debug
from sentinelhub import (
    SentinelHubRequest,
    DataCollection,
    MimeType,
    BBox,
    bbox_to_dimensions,
    CRS,
    SHConfig,
    DownloadFailedException
)

config = SHConfig()

/home/jviehweger/Documents/Projects/2023/nrt/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load evalscript

Put your evalscript into the file `script.js` and also edit your evalscript there. Do not rename the file, since it is also accessed by the html files. But first the evalscript is loaded into Python.

In [2]:
with open("script.js", "r") as file:
    evalscript = file.read()

## Specify SH Request

Here you should specify a Sentinel Hub Request.

In [3]:
bbox_coords = [46.16, -16.15, 46.51, -15.58]
crs = CRS.WGS84
resolution = 60

bbox = BBox(bbox_coords, crs)
size = bbox_to_dimensions(bbox, resolution=resolution)

request_true_color = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=("2020-01-01", "2021-01-01"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=bbox,
    size=size,
    config=config,
)

## Get data and save it locally

The next section saves `collections` locally so that preProcessScenes can be tested in the browser. It saves the data into `collections.js`.

In [ ]:
# Get collections data locally
collections = debug.get_collections(request_true_color)
# Write the data to a js file to load later
with open("./local_data/collections.js", "w") as f:
    f.write("var collections = "+collections+"\n")

You can now open [debug_preProcess.html](./debug_preProcess.html) in the browser. A completely white page should appear. When you hit F12, the debugging console should appear. In it breakpoints can be set and loggers can be specified. If there are any `console.log()` calls in the evalscript, the output will also appear.

More info on debugging javascript is available for [Chrome](https://developer.chrome.com/docs/devtools/javascript/) and [Firefox](https://firefox-source-docs.mozilla.org/devtools-user/debugger/).

If you make any changes to the `script.js` file, the debug page has to be refreshed (for example using F5). However unless you make changes to the sentinel hub request, the data which was acquired in the previous step does not have to be downloaded again.

Once the `preProcessScenes` function is working as intended, the data for the `evaluatePixel()` function can be downloaded. This is done in the next code block. Depending on the amount of data this may take quite a while, however you hopefully only have to do this once.

In [4]:
# Get example data locally
sample, scene = debug.get_sample_scene(request_true_color)
# Write the data to a js file to load later
with open("./local_data/data.js", "w") as f:
    f.writelines(["var sample = "+sample+"\n", "var scene = "+scene, "\n"])

To debug the `evaluatePixel()` function open the [debug_evaluatePixel.html](./debug_evaluatePixel.html) file in a browser and use the browser debugging tools again to fix your function.